In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(0.25)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(0.25)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(0.25)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(0.25)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 3):  # There are 2 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('kerala_bus_details.csv', index=False)


driver.quit()

NameError: name 'driver' is not defined

In [2]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,SAAM BUS,VE A/C Sleeper (2+1),21:45,09h 00m,06:45,4.3,INR 1600,9 Seats available
1,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,LEGEND HOLIDAYS,Bharat Benz A/C Semi Sleeper (2+2),21:20,08h 40m,06:00,4.9,INR 1699,24 Seats available
2,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,FAST LINE BUS,A/C Sleeper (2+1),21:15,09h 00m,06:15,4.5,INR 1499,10 Seats available
3,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,AdSam Tourist,NON AC Seater / Sleeper 2+1,22:05,09h 00m,07:05,3.7,1724,3 Seats available
4,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,CHANCHALAS TOURS AND TRAVELS,A/C Semi Sleeper (2+2),19:45,10h 15m,06:00,5.0,1598,19 Seats available
...,...,...,...,...,...,...,...,...,...,...
177,Kozhikode to Kottayam,https://www.redbus.in/bus-tickets/kozhikode-to...,Madhavi Travels,A/C Sleeper (2+1),23:30,06h 30m,06:00,4.0,INR 1199,3 Seats available
178,Kottayam to Kozhikode,https://www.redbus.in/bus-tickets/kottayam-to-...,KSRTC (Kerala) - 852,Super Fast Non AC Seater (2+3),21:00,06h 30m,03:30,3.6,INR 332,1 Seat available
179,Kottayam to Kozhikode,https://www.redbus.in/bus-tickets/kottayam-to-...,Madhavi Travels,Volvo Multi-Axle I-Shift B11R Semi Sleeper (2+2),23:59,04h 31m,04:30,4.7,INR 1499,5 Seats available
180,Kottayam to Kozhikode,https://www.redbus.in/bus-tickets/kottayam-to-...,EMERALD TRAVELS,A/C Sleeper (2+1),23:00,05h 30m,04:30,3.7,INR 1500,2 Seats available
